In [54]:
from datastreams.datastream import Streamer

import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [3]:
cd ..

/home/evan/Documents/github/Feedlot


### Load transformed cow swap dataframe

In [4]:
cow_swaps_df = pd.read_csv('data/transformed/cow_swaps.csv', low_memory=False)

In [5]:
# print memory size of cow_swap_df
print('Memory size of cow_swaps_df: ', cow_swaps_df.memory_usage().sum() / 1024**2, 'MB')

Memory size of cow_swaps_df:  63.36715126037598 MB


In [6]:
cow_swaps_df.head(5)

,timestamp,gasPrice,feeAmount,txHash,settlement,buyAmount,sellAmount,order,buyAmountEth,sellAmountEth,buyAmountUsd,sellAmountUsd,solvers,environment,name,active,sellTokenSymbol,sellTokenDecimals,buyTokenSymbol,buyTokenDecimals
0,1663907363,6062637482,1344622,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,0x0454b81f9e497fe91fc2df0e1e7420b9a5e20439b065132daab91c09de8512cf,917011543008284856417,4093394367,0x000050299d2697ae6d93d9ccec7b55dbe79c920596927402db5784e8e7ef1c9d961eed3b7d4b7c261ee5b023e5f857af5af1e700632d3cf0,3.047480,3.049075,4087.560206,4093.394367,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,USDC,6,FXS,18
1,1668043871,30744415450,37223401,0xbe9648a3caed2adc9254ea595dac1cda93af8b243b7a8c507ea33d3e4e8efbe4,0xbe9648a3caed2adc9254ea595dac1cda93af8b243b7a8c507ea33d3e4e8efbe4,8621106771373760291720,45331499213,0x0004d9673bd844cc4d44a33b1f5c1df7ef6cf5eff63ccaaf99d4442089a2deb0d1f2739ad714045be6146915275d0a2b822ec1cc636c5b20,39.399436,39.811826,44861.934154,45331.499213,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,USDC,6,FXS,18
2,1669126187,24494377559,13386244,0x66e01c20e7640f86113a8fef5b80d7393eebb01e153d33310dd94409d4adfe32,0x66e01c20e7640f86113a8fef5b80d7393eebb01e153d33310dd94409d4adfe32,1583112031078343925188,6719000000,0x00668e787a470a68eee83463fb4618ce9bedec7c50ac7ea9d2bf0d073792fe4d95861c41e7767dc737bca90b175ad51e5f7b9ada637cde92,5.996258,5.917571,6821.422376,6719.000000,0xb20b86c4e6deeb432a22d773a221898bbbd03036,prod,Gnosis_1inch,True,USDC,6,FXS,18
3,1662029515,12869344497,4808912,0xb126e3c1fdffde8ece3b6b6554ff5d6cb313ae67a692ea92f01ecfe17fb9d982,0xb126e3c1fdffde8ece3b6b6554ff5d6cb313ae67a692ea92f01ecfe17fb9d982,369820837360497071942,2138328191,0x00a3de84225174293e86edf848100dfb1123ea727209aa63bd9d72edc9838de8254725a2d9989ab4202e1f6a6560b970dc4a67da6310958d,1.380130,1.372932,2126.884281,2138.328191,0xc9ec550bea1c64d779124b23a26292cc223327b6,prod,Otex,True,USDC,6,FXS,18
4,1653185840,14600197606,20683162,0xc5c3c4cb97c972f83630f7a415ce064c9c2e17c316936d382ed3bfb54266c8e8,0xc5c3c4cb97c972f83630f7a415ce064c9c2e17c316936d382ed3bfb54266c8e8,8901647727774327086836,59366191057,0x00a5ee906f05258014318b5488bded192c21ed6a9ee7a65506a80cb459d362d8cbdd2638b5f2e0360ea43806936276ba16d0a4ab6289a3b9,29.458128,30.052786,58191.504954,59366.191057,0x6fa201c3aff9f1e4897ed14c7326cf27548d9c35,prod,Otex,False,USDC,6,FXS,18


In [7]:
# get the timestamp when each token first appeared in the cow_swaps_df
first_sell_appearance = cow_swaps_df.groupby('sellTokenSymbol')['timestamp'].min().reset_index()
first_buy_appearance = cow_swaps_df.groupby('buyTokenSymbol')['timestamp'].min().reset_index()

In [8]:
# combine appearance dataframes with the min timestamp value for each token symbol
first_appearance = pd.concat([first_sell_appearance, first_buy_appearance], axis=0)
first_appearance = first_appearance.groupby('sellTokenSymbol')['timestamp'].min().reset_index()

# rename columns
first_appearance.columns = ['tokenSymbol', 'firstAppearance']

In [9]:
first_appearance.head(5)

,tokenSymbol,firstAppearance
0,$ADS,1630035343
1,$ANRX,1629291552
2,$DG,1629051628
3,$EXAMPLE,1666229723
4,$HRIMP,1629618858


In [10]:
# check if ETH is in first_appearance tokenSymbol. ALso check that there exists tokens that don't start with $
'WETH' in first_appearance['tokenSymbol'].values

True

In [11]:
# get list of unique buy and sell token symbols
unique_buy_tokens = cow_swaps_df['buyTokenSymbol'].unique()
unique_sell_tokens = cow_swaps_df['sellTokenSymbol'].unique()

In [12]:
# find the unique tokens in both lists
unique_cow_tokens = list(set(unique_buy_tokens) & set(unique_sell_tokens))

In [13]:
# remove "nan" float value from the unique tokens list
unique_cow_tokens = [token for token in unique_cow_tokens if type(token) != float]

### Create Chainlink Streamer with DataStreams

In [14]:
endpoint = 'https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph'

In [15]:
# load streamer class
ds = Streamer(endpoint)

In [16]:
# check available query fields
ds.queryFields

[FieldPath(https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph, Query, ['prices']),
 FieldPath(https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph, Query, ['assetPairs'])]

In [17]:
asset_pairs_df = ds.runQuery(
    query_field = ds.queryDict.get('assetPairs'),
    query_size = 10000
    )

FIELD - assetPairs


In [18]:
# check how many asset pairs there are
print(f'there are {len(asset_pairs_df)} asset pairs in this chainlink subgraph node.')

# convert to list
asset_pairs_list = asset_pairs_df['assetPairs_id'].to_list()

print(f'some sample values are: {asset_pairs_list[:5]}')

there are 409 asset pairs in this chainlink subgraph node.
some sample values are: ['1INCH/ETH', '1INCH/USD', 'AAPL/USD', 'AAVE/ETH', 'AAVE/USD']


In [19]:
# get all the unique values to the right of /
denominated_asset_pairs_list = [x.split('/') for x in asset_pairs_list]

# get unique values in denominated_asset_pairs_list if in ETH or USD
eth_usd_asset_denomination_list = [x for x in denominated_asset_pairs_list if x[-1] in ['ETH', 'USD']]

In [20]:
eth_usd_asset_denomination_list[0:5]

[['1INCH', 'ETH'],
 ['1INCH', 'USD'],
 ['AAPL', 'USD'],
 ['AAVE', 'ETH'],
 ['AAVE', 'USD']]

In [21]:
# get assets denominated in USD
usd_asset_denomination_list = [x for x in eth_usd_asset_denomination_list if x[-1] == 'USD']

# cross reference usd assets with cow tokens
cow_usd_asset_denomination_list = [x for x in usd_asset_denomination_list if x[0] in unique_cow_tokens]

In [97]:
# sample output
cow_usd_asset_denomination_list[0:5]

[['1INCH', 'USD'],
 ['AAVE', 'USD'],
 ['ACH', 'USD'],
 ['ADX', 'USD'],
 ['AKRO', 'USD']]

### Create custom search vals parameters based on tokens in cow_swaps dataset

In [22]:
# make a token search pair list for the cow tokens
token_search_pair_list = [x[0] + '/' + x[1] for x in cow_usd_asset_denomination_list]

In [24]:
# ETH and BTC are not in unique_tokens, but WETH and WBTC are.
'WETH' in unique_cow_tokens

True

In [25]:
# manually add ETH/USD and BTC/USD into the token_search_pair_list
token_search_pair_list.append('ETH/USD')
token_search_pair_list.append('BTC/USD')

In [26]:
print(f'there are {len(token_search_pair_list)} token pairs denominated in USD. Sample output is: {token_search_pair_list[0:5]}')

there are 106 token pairs denominated in USD. Sample output is: ['1INCH/USD', 'AAVE/USD', 'ACH/USD', 'ADX/USD', 'AKRO/USD']


In [32]:
# COMMENT OUT BECAUSE I DONT WANT TO DEAL WITH INDEX ERROR. JUST CHOOSING MIN TIMESTAMP OUT OF EVERY VALUE

# # for token in token_search_pair_list, get the earliest dataframe that the token first appeared in via earliest_appearance
# for token in token_search_pair_list:
#     earliest_appearance = first_appearance.loc[first_appearance['tokenSymbol'] == token.split('/')[0]]['firstAppearance'].values[0]
#     print(f'the earliest appearance of {token} is {earliest_appearance}')

the earliest appearance of 1INCH/USD is 1628253224
the earliest appearance of AAVE/USD is 1628283733
the earliest appearance of ACH/USD is 1628684399
the earliest appearance of ADX/USD is 1636820535
the earliest appearance of AKRO/USD is 1629568603
the earliest appearance of ALBT/USD is 1628960115
the earliest appearance of ALCX/USD is 1628739128
the earliest appearance of AMPL/USD is 1628720433
the earliest appearance of AMP/USD is 1628776245
the earliest appearance of ANKR/USD is 1629232721
the earliest appearance of APE/USD is 1647522023
the earliest appearance of AUDIO/USD is 1628869575
the earliest appearance of BADGER/USD is 1628735791
the earliest appearance of BAL/USD is 1628254173
the earliest appearance of BAND/USD is 1628799456
the earliest appearance of BAT/USD is 1628679675
the earliest appearance of BICO/USD is 1638430869
the earliest appearance of BIT/USD is 1629174652
the earliest appearance of BNT/USD is 1628146744
the earliest appearance of BOR/USD is 1636980956
the e

IndexError: index 0 is out of bounds for axis 0 with size 0

In [43]:
# get smallest first_appearance value
earliest_appearance = first_appearance['firstAppearance'].min()

In [46]:
keys = ['timestamp_lt', 'assetPair']
initial_vals = [1673800000, 'ETH/USD']

In [47]:
price_dfs_list = ds.runSameQuerySearch(
    fieldParam = ds.queryDict.get('prices'),
    keys = keys, 
    values = initial_vals,
    searchKey = keys[1],
    searchVals = token_search_pair_list,
    query_size = 10000000
    )

search_dict: {'timestamp_lt': 1673800000, 'assetPair': 'ETH/USD'}
val: 1INCH/USD
FIELD - prices
Filter based on these values: {'timestamp_lt': 1673800000, 'assetPair': '1INCH/USD'}
1INCH/USD                   prices_id prices_assetPair_id  prices_timestamp  prices_price
0      1INCH/USD/0x63c40253           1INCH/USD        1673790035      49746305
1      1INCH/USD/0x63c3ffa7           1INCH/USD        1673789351      49164742
2      1INCH/USD/0x63c3fe7b           1INCH/USD        1673789051      48662978
3      1INCH/USD/0x63c3c287           1INCH/USD        1673773703      48152059
4      1INCH/USD/0x63c3b34b           1INCH/USD        1673769803      48659796
...                     ...                 ...               ...           ...
14929  1INCH/USD/0x6066cb91           1INCH/USD        1617349521     448793445
14930  1INCH/USD/0x6066ca8e           1INCH/USD        1617349262     448729456
14931  1INCH/USD/0x6066b91e           1INCH/USD        1617344798     448758913
14932  1I

In [50]:
price_dfs_list[0].head(5)

,prices_id,prices_assetPair_id,prices_timestamp,prices_price
0,1INCH/USD/0x63c40253,1INCH/USD,1673790035,49746305
1,1INCH/USD/0x63c3ffa7,1INCH/USD,1673789351,49164742
2,1INCH/USD/0x63c3fe7b,1INCH/USD,1673789051,48662978
3,1INCH/USD/0x63c3c287,1INCH/USD,1673773703,48152059
4,1INCH/USD/0x63c3b34b,1INCH/USD,1673769803,48659796


In [52]:
# get prices_id to the left of /
name = price_dfs_list[0]['prices_id'].apply(lambda x: x.split('/')[0])[0]

In [55]:
# make directory if it doesn't exist
if not os.path.exists(f'data/chainlink_prices'):
    os.makedirs(f'data/chainlink_prices')

# save each dataframe to a csv file
for df in price_dfs_list:
    name = df['prices_id'].apply(lambda x: x.split('/')[0])[0]
    df.to_csv(f'data/chainlink_prices/{name}.csv', index=False)

In [ ]:
print(len(price_dfs_list))

106
